### Set Up

In [2]:
"""
PRAW Docs: https://praw.readthedocs.io/en/stable/tutorials/comments.html
Sentiment Analysis Using HuggingFace: https://huggingface.co/blog/sentiment-analysis-python
Cultural Analytics with Python: https://melaniewalsh.github.io/Intro-Cultural-Analytics/04-Data-Collection/14-Reddit-Data.html
"""

# Set up Pandas

import pandas as pd
pd.set_option('max_colwidth', 500)


# Set up PRAW with athentication

import praw
reddit = praw.Reddit(
    client_id = "QErgebb-REIyaM6wsoQ-Nw",
    client_secret = "PPuRFlKap6UQ4D5f9tYi_pvY68ePkA",
    username = "Ok_Scientist2546",
    password = "EZ8y@'ctT!f4A%L",
    user_agent = "Praw-test"
)

/Users/racliu/Documents/praw/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Determine Available Attributes of a Submission object
import pprint

submission = reddit.submission("14828yd")
# print(submission.title)  # to make it non-lazy
# pprint.pprint(vars(submission))

# # Determine Available Attributes of a Comment object
# comment = list(submission.comments)[0]
# print(comment.body)  
# pprint.pprint(vars(comment))


Very cool story
{'_fetched': True,
 '_reddit': <praw.reddit.Reddit object at 0x106672a00>,
 '_replies': <praw.models.comment_forest.CommentForest object at 0x116fb5a90>,
 '_submission': Submission(id='14828yd'),
 'all_awardings': [],
 'approved_at_utc': None,
 'approved_by': None,
 'archived': False,
 'associated_award': None,
 'author': Redditor(name='letsgoabbey'),
 'author_flair_background_color': None,
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_template_id': None,
 'author_flair_text': None,
 'author_flair_text_color': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_ft20qahn',
 'author_is_blocked': False,
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'banned_at_utc': None,
 'banned_by': None,
 'body': 'Very cool story',
 'body_html': '<div class="md"><p>Very cool story</p>\n</div>',
 'can_gild': True,
 'can_mod_post': False,
 'collapsed': False,
 'collapsed_because_crowd_control': None,
 'collapsed_reason'

### Build Submissions Dataframe

In [62]:
### FUNCTIONS ###

from datetime import datetime

def pull_submissions(num_subs: int, sub_name: str, sort: str):
    """
    Gets key details about num_subs number of submissions on a particular subreddit sub_name. 

    Inputs:
        num_subs [int]: the number of submissions to pull
        sub_name [str]: subreddit name without the r/, i.e., "scarystories"
        sort [str]: the way to sort the subreddit, i.e. by "controversial," "gilded," "hot," "new," "rising," or "top".

    
    Returns:
        [List[Dict[9 items]]]: a list of dictionaries, one for each submission in the specified subreddit 
    """

    res = []
    subreddit = reddit.subreddit(sub_name)

    SORTED_SUBMISSIONS = {"hot": subreddit.hot(limit=num_subs), 
                          "controversial": subreddit.controversial(limit=num_subs), 
                          "gilded": subreddit.gilded(limit=num_subs),
                          "top": subreddit.top(limit=num_subs),
                          "new": subreddit.new(limit=num_subs),
                          "rising": subreddit.rising(limit=num_subs)
                          }
    
    for submission in SORTED_SUBMISSIONS[sort]:
        story = {}
        story["title"] = submission.title
        story["submission_id"] = submission.id
        story["score"] = submission.score
        story["url"] = submission.url
        story["author"] = submission.author.name
        story["text"] = submission.selftext
        story["subreddit"] = submission.subreddit
        story["num_comments"] = submission.num_comments
        story["date_created"] = datetime.fromtimestamp(submission.created_utc)
        res.append(story)
    
    return res


In [63]:
# write submissions to csv and import as DF
import csv

hot_ten_stories = pull_submissions(num_subs=10, sub_name="scarystories", sort="hot")
fieldnames = hot_ten_stories[0].keys()

with open('submissions.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(hot_ten_stories)

# Import CSV data into Pandas as a DF

submissions_df = pd.read_csv("submissions.csv", delimiter=',', encoding="UTF-8")
submissions_df

,title,id,score,url,author,text,subreddit,num_comments,date_created
0,I found out I have a stalker through TikTok Live,149d6my,2,https://www.reddit.com/r/scarystories/comments/149d6my/i_found_out_i_have_a_stalker_through_tiktok_live/,IndigoCreepy,"One night, unable to sleep, I mindlessly scrolled through TikTok, hoping to find something that would distract me from my restlessness. Among the countless TikTok Live streams, one particular stream caught my attention. The title simply read, ""A Midnight Stroll."" Intrigued, I tapped on it, unknowingly opening the door to a world of horror. The stream depicted a man silently traversing through a desolate forest. The eerie atmosphere was suffocating, with darkness seemingly swallowing the lone...",scarystories,0,2023-06-14 13:05:48
1,STAY INSIDE : SURVIVING THE UNSEEN APOCALYPSE,149elve,1,https://www.reddit.com/r/scarystories/comments/149elve/stay_inside_surviving_the_unseen_apocalypse/,Scared_Sorbet4483,"EMERGENCY ALERT.\n\nPlease be advised, this is not a drill. As of 12:34 EST, there has been a nationwide security breach. All citizens must stay in their homes, avoid all windows, and keep all entrances securely locked. Do not, under any circumstances, answer your door. If you hear voices of known individuals, disregard them. They are not what they seem. Stay tuned for further updates.\n\nI stared at the notification on my phone. Nationwide security breach? Was this some kind of prank? But e...",scarystories,0,2023-06-14 14:03:09
2,I love you most,148tkg7,25,https://www.reddit.com/r/scarystories/comments/148tkg7/i_love_you_most/,kokkaschloe,"She was the love of my life. I wish I could express to you just how much I loved her, and she knew it. She was so kind, funny, interesting, and gorgeous. I knew she loved me, just as much as I loved her. She loved Marvel movies, especially Spider-Man, cooking, and swimming. She was such a fast swimmer. When we went to the lake, I would pretend to chase after her and she would swim away, screaming and laughing. I never saw her laugh again. She became depressed after she lost her mother. A mur...",scarystories,3,2023-06-13 20:10:49
3,The mcnightmare,149dze9,1,https://www.reddit.com/r/scarystories/comments/149dze9/the_mcnightmare/,Scared_Sorbet4483,"I have always been a loyal customer of McDonald's, indulging in their greasy delights without a second thought. Little did I know that my seemingly innocent visits to the golden arches would soon turn into a waking nightmare.\n\nIt started with a chance encounter. As I sat alone, enjoying my meal, I noticed a figure in the corner of my eye. A man, nondescript in appearance, sat at a nearby table, fixated on me with an unsettling intensity. Unease crept through my veins, but I brushed it off ...",scarystories,0,2023-06-14 13:38:00
4,House On The Hill,14954og,3,https://www.reddit.com/r/scarystories/comments/14954og/house_on_the_hill/,Verrgasm,"There's an old abandoned house perched atop a stubby hill not too far from where I grew up. The greenish outer paint all flaked and its untended yard long since overgrown with weeds, it looms above the rest and looks down upon the surrounding modern homes littered around on all sides. A relic of a time gone past, many in the local community have always considered it an eyesore. A problem to be dealt with in order to pave the way for something new, something worthwhile. It's sat empty for man...",scarystories,0,2023-06-14 07:11:24
5,The Doll Maker,14925ut,4,https://www.reddit.com/r/scarystories/comments/14925ut/the_doll_maker/,GrimmInDarkness,"In a little village, tucked away from the rest of the world lived a doll maker up on a hill, completely isolated from everyone else. \n \n\n \nThe rumor goes that the doll maker can make any type of doll that you wish, but the only thing he will not do is bring a loved one back to life. Since he had once dabbled in black magic, breaking the rules that was once taught to him by his master. He had brought back to life a loved one

### Build Comments Dataframe

In [4]:
def pull_comments(subreddit_id, amount="all"):
    """
    Pull all or top level comments from a certain reddit submission.

    Inputs:
        subreddit_id [str]: the subreddit id of subreddit you want to pull from
        amount [str]: how many comments to pull, all comments or only top level comments. 
            By default, this variable has value "all"


    Returns: 
        [List[Dict[8 items]]]: a list of comments from a single submission with the comment details
    """

    submission = reddit.submission(subreddit_id)

    # Select top level comments or all comments 
    submission.comments.replace_more(limit=None)
    if amount == "top_level":
        comments = []
        for top_level_comment in submission.comments:
            comments.append(top_level_comment) 
    else:
        comments = submission.comments.list()

    # Return List of dictionaries with comment details
    res = []
    for comment in comments:
        new_comment = {}
        new_comment["text"] = comment.body.replace("’", "'").replace("\n", " ")
        new_comment["author"] = comment.author.name
        new_comment["score"] = comment.score
        new_comment['comment_id'] = comment.id
        new_comment["is_op"] = comment.is_submitter
        new_comment["submission_id"] = comment._submission.id
        new_comment["subreddit"] = comment.subreddit_name_prefixed
        new_comment["subreddit_id"] = comment.subreddit_id
        res.append(new_comment)
    return res


In [5]:
# Write newly-pulled comments from r/scarystories to comments.csv file

import csv

new_story = pull_comments(subreddit_id="14828yd")
fieldnames = new_story[0].keys()

with open('comments.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(new_story)

# Import CSV data into a Pandas DF

comments_df = pd.read_csv("comments.csv", delimiter=',', encoding="UTF-8")
comments_df

,text,author,score,is_op,submission_id,subreddit,subreddit_id,comment_id
0,Very cool story,letsgoabbey,5,False,14828yd,r/scarystories,t5_2rrzd,jnyiaea
1,She's using her skills.,monkner,1,False,14828yd,r/scarystories,t5_2rrzd,jnysmzi
2,Nice,Deb6691,1,False,14828yd,r/scarystories,t5_2rrzd,jnyvqfc
3,"Kiss of death, nice.",TXJOEMAMA,1,False,14828yd,r/scarystories,t5_2rrzd,jo01y4x
4,Well done. Engaging story.,Quickhidemeplease,1,False,14828yd,r/scarystories,t5_2rrzd,jo15o93
5,holy crap!! i love this sm,julesisgayasf,1,False,14828yd,r/scarystories,t5_2rrzd,jo1icjb
6,Slow clap. Well done.,Impossible_Balance11,1,False,14828yd,r/scarystories,t5_2rrzd,jo2lpzc
7,Making lemonade from lemons😉,Stranger_at_Night,1,False,14828yd,r/scarystories,t5_2rrzd,jo47rsf
8,Nice!,ThrowAway072343,0,False,14828yd,r/scarystories,t5_2rrzd,jny4dqi
9,"Thank you, I'm glad you enjoyed it.",1000andonenites,3,True,14828yd,r/scarystories,t5_2rrzd,jnz9x5y


In [ ]:
# Only pull between a certain time range

current_time = datetime.datetime.now(datetime.timezone.utc).timestamp()
one_week_ago = current_time - 604800
subreddit = reddit.subreddit("scarystories")

for submission in subreddit.new(limit=25):
	if submission.created_utc >= one_week_ago and < another_time_range:
		# do stuff
		pass

### Master Checklist

- [ ] pulling only from certain time range: 
you can use pushshift to grab the bulk posts from the specific date range and then parse the specific posts using their retrieved IDs with PRAW if needed.
- [ ] add work to Github
